In [1]:

#? python train_autoencoder.py -h
#? python train_autoencoder.py steerable3DCNN trainings/001 500

#? python train_forecaster.py -h
#? python train_forecaster.py steerable3DCNN trainingsFC/001 trainingsAE/001 500

#? python evaluate.py -h
#? python evaluate.py D4cnn training1 [-eval_performance | 
#?                                     -eval_performance_per_sim |
#?                                     -eval_performance_per_channel |
#?                                     -eval_performance_per_height |
#?                                     -check_equivariance |
#?                                     -animate |
#?                                     -compute_latent_sensitivity]

#? tensorboard --logdir runs


# 3D Rayleigh-Bénard: Equivariant Forecasting

In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys

sys.path.append('..')

from IPython.display import Video, display
from ipywidgets import interact_manual, FloatSlider, IntSlider, BoundedIntText, Dropdown, SelectMultiple, Checkbox, Textarea
import glob
import json

from utils.model_building import build_autoencoder, build_forecaster, build_and_load_trained_model
from utils.evaluation import load_latent_sensitivity
from utils import visualization

2025-02-11 02:14:59.448060: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-11 02:14:59.480146: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-11 02:15:00.014662: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Hyperparameter Selection

### Autoencoder

In [ ]:
model_type_widget = Dropdown(options=['steerableCNN', 'steerable3DCNN', 'CNN', '3DCNN'], value='steerableCNN')
simulation_widget = Dropdown(options=glob.glob('*.h5', root_dir='../data/datasets'))
rots_widget = IntSlider(min=4, max=16, step=4, value=4)
flips_widget = Checkbox(value=True)
encoder_channels_widget = Textarea()
pooling_layers_widget = Textarea(value='y,y,y,y')
latent_channels_widget = BoundedIntText(min=1, max=1000, value=32)
h_ksize_widget = Dropdown(options=[3, 5], value=5)
v_ksize_widget = Dropdown(options=[3, 5], value=5)
latent_h_ksize_widget = Dropdown(options=[3, 5], value=3)
latent_v_ksize_widget = Dropdown(options=[3, 5], value=3)
drop_rate_widget = FloatSlider(min=0, max=1, value=0.2)
nonlinearity_widget = Dropdown(options=['ReLU', 'ELU', 'LeakyReLU'], value='ELU')

def set_default_channels(*args):
    default_channels = {'steerableCNN': "6,12,23,47",
                        'steerable3DCNN': "24,48,93,186",
                        'CNN': "10,20,40,80",
                        '3DCNN': "24,47,93,186"}[model_type_widget.value]
    encoder_channels_widget.value = default_channels
        
model_type_widget.observe(set_default_channels)


@interact_manual.options(manual_name="Build model")(model_type=model_type_widget, simulation=simulation_widget,
                                                    rots=rots_widget, flips=flips_widget, encoder_channels=encoder_channels_widget,
                                                    pooling_layers=pooling_layers_widget, latent_channels=latent_channels_widget, 
                                                    h_ksize=h_ksize_widget, v_ksize=v_ksize_widget, 
                                                    latent_h_ksize=latent_h_ksize_widget, 
                                                    latent_v_ksize=latent_v_ksize_widget, 
                                                    drop_rate=drop_rate_widget, 
                                                    nonlinearity=nonlinearity_widget)
def show_patterns(model_type, simulation, rots, flips, encoder_channels, pooling_layers, latent_channels, h_ksize, v_ksize, 
                  latent_h_ksize, latent_v_ksize, drop_rate, nonlinearity):
    encoder_channels = [int(c.strip()) for c in encoder_channels.split(',')]
    true_strings = ['1', 't', 'y', 'true', 'yes', 'p']
    pool_layers = [p.strip().lower() in true_strings for p in pooling_layers.split(',')]
    assert len(encoder_channels) == len(pool_layers)
    
    hps = {
        'simulation_name': simulation,
        'rots': rots,
        'flips': flips,
        'encoder_channels': encoder_channels,
        'latent_channels': latent_channels,
        'h_kernel_size': h_ksize,
        'v_kernel_size': v_ksize,
        'latent_h_kernel_size': latent_h_ksize,
        'latent_v_kernel_size': latent_v_ksize,
        'drop_rate': drop_rate,
        'nonlinearity': nonlinearity,
        'pool_layers': pool_layers
    }
    
    model = build_autoencoder(model_type, **hps)

    model.summary()

interactive(children=(Dropdown(description='model_type', options=('steerableCNN', 'steerable3DCNN', 'CNN', '3D…

interactive(children=(Dropdown(description='model_type', options=('steerableCNN', 'steerable3DCNN', 'CNN', '3D…

### Forecaster

In [3]:
model_type_widget = Dropdown(options=['steerableCNN', '3DCNN'], value='steerableCNN')
autoencoder_widget = Dropdown(options=[])
rots_widget = IntSlider(min=4, max=16, step=4, value=4)
flips_widget = Checkbox(value=True)
lstm_channels_widget = Textarea(value='8,8')
residual_connection_widget = Checkbox(value=True)
h_ksize_widget = Dropdown(options=[3, 5], value=3)
v_ksize_widget = Dropdown(options=[3, 5], value=3)
drop_rate_widget = FloatSlider(min=0, max=1, value=0.2)
recurrent_drop_rate_widget = FloatSlider(min=0, max=1, value=0)
nonlinearity_widget = Dropdown(options=['ReLU', 'tanh'], value='tanh')
include_autoencoder_widget = Checkbox(value=False)
use_lstm_encoder_widget = Checkbox(value=True)
    
def update_autoencoders(*args):
    model_name = {'steerableCNN': f'{"D" if flips_widget.value else "C"}{rots_widget.value}cnn',
                  'steerable3DCNN': f'3D-{"D" if flips_widget.value else "C"}{rots_widget.value}cnn',
                  'CNN': 'cnn',
                  '3DCNN': '3Dcnn'}[model_type_widget.value]
    trained_autoencoders = [os.path.join(*path.split('/')[:-1]) 
                          for path in glob.glob(f'{model_name}/**/epoch*.tar', root_dir='./trained_models/AE', recursive=True)]
    autoencoder_widget.options = sorted(trained_autoencoders)
    autoencoder_widget.value = autoencoder_widget.options[0]
        
model_type_widget.observe(update_autoencoders)


@interact_manual.options(manual_name="Build model")(model_type=model_type_widget, autoencoder=autoencoder_widget,
                                                    rots=rots_widget, flips=flips_widget, lstm_channels=lstm_channels_widget,
                                                    h_ksize=h_ksize_widget, v_ksize=v_ksize_widget, 
                                                    drop_rate=drop_rate_widget, recurrent_drop_rate=recurrent_drop_rate_widget, 
                                                    nonlinearity=nonlinearity_widget, residual_connection=residual_connection_widget,
                                                    include_autoencoder=include_autoencoder_widget,
                                                    use_lstm_encoder=use_lstm_encoder_widget)
def show_patterns(model_type, autoencoder, rots, flips, lstm_channels, h_ksize, v_ksize, drop_rate, recurrent_drop_rate,
                  nonlinearity, residual_connection, include_autoencoder, use_lstm_encoder):
    ae_model_name = os.path.join(*autoencoder.split('/')[:2])
    ae_train_name = os.path.join(*autoencoder.split('/')[2:])
    
    lstm_channels = [int(c.strip()) for c in lstm_channels.split(',')]
    
    hps = {
    'model_type': model_type,
    'ae_model_name': ae_model_name,
    'ae_train_name': ae_train_name,
    'h_kernel_size': h_ksize,
    'v_kernel_size': v_ksize,
    'drop_rate': drop_rate,
    'recurrent_drop_rate': recurrent_drop_rate,
    'nonlinearity': nonlinearity,
    'flips': flips,
    'rots': rots,
    'lstm_channels': lstm_channels,
    'parallel_ops': True,
    'residual_connection': residual_connection,
    'include_autoencoder': include_autoencoder,
    'use_lstm_encoder': use_lstm_encoder
}
    
    model = build_forecaster(models_dir='./trained_models', **hps)

    model.summary()

interactive(children=(Dropdown(description='model_type', options=('steerableCNN', '3DCNN'), value='steerableCN…

## Trained Model Summaries

In [6]:
trained_models_w_epoch = [os.path.join(*path.split('/')[:-1]) 
                          for path in glob.glob('**/epoch*.tar', root_dir='./trained_models', recursive=True)]
trained_models_widget = Dropdown(options=sorted(trained_models_w_epoch))

@interact_manual.options(manual_name="Build model")(trained_model=trained_models_widget)
def show_summary(trained_model, show_summary: bool = True, show_hps: bool = True):
    model_name = os.path.join(*trained_model.split('/')[:2])
    train_name = os.path.join(*trained_model.split('/')[2:])
    
    if show_summary:
        model = build_and_load_trained_model('trained_models', model_name, train_name, epoch=-1)
        model.summary()
    
    if show_hps:
        print('\nHyperparameters:')
        hp_file = os.path.join('trained_models', model_name, train_name, 'hyperparameters.json')
        if os.path.isfile(hp_file):
            with open(hp_file, 'r') as f:
                hps = json.load(f)
                print(json.dumps(hps, indent=4)[2:-2])

interactive(children=(Dropdown(description='trained_model', options=('AE/3D-D4cnn/hp_search/001_lrschedule_lr1…

## Evaluation

### Loss Evolution During Training

In [ ]:
trained_models_w_log = [os.path.join(*path.split('/')[:-1]) 
                        for path in glob.glob('**/log.json', root_dir='./trained_models', recursive=True)]

trained_models_widget = SelectMultiple(options=sorted(trained_models_w_log))
smoothing_widget = FloatSlider(min=0, max=0.9, step=0.05)
x_axis_widget = Dropdown(options=['time', 'epochs'], value='epochs')

@interact_manual.options(manual_name="Visualize Loss")(trained_models=trained_models_widget, smoothing=smoothing_widget, x_axis=x_axis_widget)
def show_loss(trained_models, two_plots=True, log_scale=False, remove_outliers=True, x_axis='epochs', smoothing=0):
    if not trained_models: 
        print('Please select a trained model')
        return
    
    model_names = [os.path.join(*path.split('/')[:2]) for path in trained_models]
    train_names = [os.path.join(*path.split('/')[2:]) for path in trained_models]
    
    visualization.plot_loss_history('trained_models', model_names, train_names, two_plots, log_scale, 
                                    x_axis=='time', remove_outliers, smoothing)

interactive(children=(SelectMultiple(description='trained_models', options=('AE/3D-D4cnn/hp_search/001_lrsched…

### Test Performance

In [ ]:
trained_models_w_performance = [os.path.join(*path.split('/')[:-1]) 
                                for path in glob.glob('**/performance.json', root_dir='./results', recursive=True)]
trained_models_widget = SelectMultiple(options=sorted(trained_models_w_performance))
metric_widget = Dropdown(options=['MSE', 'RMSE', 'MAE'], value='RMSE')

@interact_manual.options(manual_name="Plot performance")(trained_models=trained_models_widget, metric=metric_widget)
def show_performance(trained_models, metric):
    if len(trained_models) < 2: 
        print('Please select at least 2 trained model')
        return
    
    model_names = [os.path.join(*path.split('/')[:2]) for path in trained_models]
    train_names = [os.path.join(*path.split('/')[2:]) for path in trained_models]
    
    visualization.plot_performance('./results', model_names, train_names, metric.lower())

interactive(children=(SelectMultiple(description='trained_models', options=('AE/3Dcnn/hp_search/008_3layer_lc3…

### Autoregressive Test Performance

In [30]:
trained_models_w_performance = [os.path.join(*path.split('/')[:-1]) 
                                for path in glob.glob('**/autoregressive_performance.json', root_dir='./results', recursive=True)]
trained_models_widget = SelectMultiple(options=sorted(trained_models_w_performance))
metric_widget = Dropdown(options=['MSE', 'RMSE', 'MAE'], value='RMSE')
show_train_widget = Checkbox(value=False)
show_bounds_widget = Checkbox(value=True)
median_widget = Checkbox(value=False)

#! there is some bug, see median of FC/3Dcnn/test/001

@interact_manual.options(manual_name="Plot performance")(trained_models=trained_models_widget, metric=metric_widget,
                                                         show_train=show_train_widget, show_bounds=show_bounds_widget,
                                                         median=median_widget)
def show_performance(trained_models, metric, show_train, show_bounds, median):
    if len(trained_models) < 1: 
        print('Please select a trained model')
        return
    model_names = [os.path.join(*path.split('/')[:2]) for path in trained_models]
    train_names = [os.path.join(*path.split('/')[2:]) for path in trained_models]
    
    visualization.plot_autoregressive_performance('./results', model_names, train_names, metric.lower(), 
                                                  show_train, show_bounds, median)

interactive(children=(SelectMultiple(description='trained_models', options=('FC/3Dcnn/test/001', 'FC/3Dcnn/tes…

### Test Performance per Simulation

In [ ]:
trained_models_w_performance = [os.path.join(*path.split('/')[:-1]) 
                                for path in glob.glob('**/performance_per_sim.json', 
                                                      root_dir='./results', recursive=True)]
trained_models_widget = SelectMultiple(options=sorted(trained_models_w_performance))
metric_widget = Dropdown(options=['MSE', 'RMSE', 'MAE'], value='RMSE')

@interact_manual.options(manual_name="Plot performance")(trained_models=trained_models_widget, metric=metric_widget)
def show_performance(trained_models, metric):
    if not trained_models: 
        print('Please select a trained model')
        return
    
    model_names = [os.path.join(*path.split('/')[:2]) for path in trained_models]
    train_names = [os.path.join(*path.split('/')[2:]) for path in trained_models]
    
    visualization.plot_performance_per_sim('./results', model_names, train_names, metric.lower())

interactive(children=(SelectMultiple(description='trained_models', options=('AE/3Dcnn/hp_search/001_lr1e-3', '…

### Test Performance per Channel

In [ ]:
trained_models_w_performance = [os.path.join(*path.split('/')[:-1]) 
                                for path in glob.glob('**/performance_per_channel.json', 
                                                      root_dir='./results', recursive=True)]
trained_models_widget = SelectMultiple(options=sorted(trained_models_w_performance))
metric_widget = Dropdown(options=['MSE', 'RMSE', 'MAE'], value='RMSE')

@interact_manual.options(manual_name="Plot performance")(trained_models=trained_models_widget, metric=metric_widget)
def show_performance(trained_models, metric):
    if not trained_models: 
        print('Please select a trained model')
        return
    
    model_names = [os.path.join(*path.split('/')[:2]) for path in trained_models]
    train_names = [os.path.join(*path.split('/')[2:]) for path in trained_models]
    
    visualization.plot_performance_per_channel('./results', model_names, train_names, metric.lower())

interactive(children=(SelectMultiple(description='trained_models', options=('AE/D4cnn/hp_search/012_no-pool4_l…

### Test Performance per Height

In [ ]:
trained_models_w_performance = [os.path.join(*path.split('/')[:-1]) 
                                for path in glob.glob('**/performance_per_height.json', 
                                                      root_dir='./results', recursive=True)]
trained_models_widget = SelectMultiple(options=sorted(trained_models_w_performance))
metric_widget = Dropdown(options=['MSE', 'RMSE', 'MAE'], value='RMSE')
channel_widget = Dropdown(options=['all', 't', 'u', 'v', 'w'], value='t')

@interact_manual.options(manual_name="Plot performance")(trained_models=trained_models_widget, metric=metric_widget,
                                                         channel=channel_widget)
def show_performance(trained_models, metric, channel):
    if not trained_models: 
        print('Please select a trained model')
        return
    
    model_names = [os.path.join(*path.split('/')[:2]) for path in trained_models]
    train_names = [os.path.join(*path.split('/')[2:]) for path in trained_models]
    channel = "tuvw".index(channel) if channel != 'all' else None
    visualization.plot_performance_per_height('./results', model_names, train_names, metric.lower(), channel)

interactive(children=(SelectMultiple(description='trained_models', options=('AE/D4cnn/hp_search/012_no-pool4_l…

### Test Performance per Latent Size

In [12]:
trained_models_w_performance = [os.path.join(*path.split('/')[:-1]) 
                                for path in glob.glob('AE/**/performance.json', 
                                                      root_dir='./results/', recursive=True)]
trained_models_widget = SelectMultiple(options=sorted(trained_models_w_performance))
metric_widget = Dropdown(options=['MSE', 'RMSE', 'MAE'], value='RMSE')

@interact_manual.options(manual_name="Plot performance")(trained_models=trained_models_widget, metric=metric_widget)
def show_performance(trained_models, metric):
    if len(trained_models) < 2: 
        print('Please select at least 2 trained models')
        return
    
    model_names = [os.path.join(*path.split('/')[:2]) for path in trained_models]
    train_names = [os.path.join(*path.split('/')[2:]) for path in trained_models]
    visualization.plot_performance_per_hp('./trained_models', './results', model_names, train_names, 
                                          metric.lower(), 'latent_size', 'relative latent size (%)')

interactive(children=(SelectMultiple(description='trained_models', options=('AE/3D-D4cnn/hp_search/001_lrsched…

### Test Performance per Parameter Count

In [11]:
trained_models_w_performance = [os.path.join(*path.split('/')[:-1]) 
                                for path in glob.glob('**/performance.json', 
                                                      root_dir='./results', recursive=True)]
trained_models_widget = SelectMultiple(options=sorted(trained_models_w_performance))
metric_widget = Dropdown(options=['MSE', 'RMSE', 'MAE'], value='RMSE')

@interact_manual.options(manual_name="Plot performance")(trained_models=trained_models_widget, metric=metric_widget)
def show_performance(trained_models, metric):
    if len(trained_models) < 2: 
        print('Please select at least 2 trained models')
        return
    
    model_names = [os.path.join(*path.split('/')[:2]) for path in trained_models]
    train_names = [os.path.join(*path.split('/')[2:]) for path in trained_models]
    visualization.plot_performance_per_hp('./trained_models', './results', model_names, train_names, 
                                          metric.lower(), 'parameters')

interactive(children=(SelectMultiple(description='trained_models', options=('AE/3D-D4cnn/hp_search/001_lrsched…

### Model Output Animation

In [17]:
trained_models_w_animation = [os.path.join(*path.split('/')[:-1]) 
                              for path in glob.glob('**/animations', root_dir='./results', recursive=True)]
trained_model_widget = Dropdown(options=sorted(trained_models_w_animation))
feature_widget = Dropdown(options=['t', 'u', 'v', 'w'])
dim_widget = Dropdown(options=['width', 'depth', 'height'])

@interact_manual.options(manual_name="Show animation")(trained_model=trained_model_widget, feature=feature_widget, dim=dim_widget)
def show_animation(trained_model, feature='t', dim='height'):    
    model_name = os.path.join(*trained_model.split('/')[:2])
    train_name = os.path.join(*trained_model.split('/')[2:])
    
    anim_file = os.path.join('.', 'results', model_name, train_name, 'animations', feature, f'{dim}.mp4')
    
    if not os.path.isfile(anim_file):
        print('There is no animation for this selection')
    else:
        display(Video(anim_file))

interactive(children=(Dropdown(description='trained_model', options=('AE/D4cnn/hp_search/002_lr1e-4',), value=…

### Latent Space Visualization (Input Sensitivity)

In [18]:
trained_models_w_sensitivity = [os.path.join(*path.split('/')[:-1]) 
                                for path in glob.glob('**/latent_sensitivity.pt', root_dir='./results/AE', recursive=True)]
trained_model_widget = Dropdown(options=sorted(trained_models_w_sensitivity))
feature_widget = Dropdown(options=['t', 'u', 'v', 'w'], value='t')
dim_widget = Dropdown(options=['width', 'depth', 'height'], value='height')
slice_widget = IntSlider(min=0, max=31, value=0)
num_patterns_widget = BoundedIntText(min=1, max=576, value=50)
cols_widget = IntSlider(min=1, max=10, value=10)


def update_slice_range(*args):
    if dim_widget.value == 'height':
        slice_widget.max = 31
        slice_widget.value = min(31, slice_widget.value)
    else:
        slice_widget.max = 47
        
dim_widget.observe(update_slice_range)

cached, cached_avg_sensitivity, cached_avg_abs_sensitivity = None, None, None

@interact_manual.options(manual_name="Show patterns")(trained_model=trained_model_widget, feature=feature_widget, 
                                                      dim=dim_widget, slice=slice_widget, num_patterns=num_patterns_widget, 
                                                      cols=cols_widget)
def show_patterns(trained_model, feature='t', dim='height', slice=16, abs_sensitivity=False, 
                  contour=False, num_patterns=25, cols=5):
    global cached, cached_avg_sensitivity, cached_avg_abs_sensitivity
    
    model_name = trained_model.split('/')[0]
    train_name = os.path.join(*trained_model.split('/')[1:])
    
    if cached == trained_model:
        avg_sensitivity, avg_abs_sensitivity = cached_avg_sensitivity, cached_avg_abs_sensitivity
    else:
        sensitivity_dir = os.path.join('.', 'results/AE', model_name, train_name)
        avg_sensitivity, avg_abs_sensitivity, n = load_latent_sensitivity(sensitivity_dir, 'latent_sensitivity')
        cached = trained_model
        cached_avg_sensitivity, cached_avg_abs_sensitivity = avg_sensitivity, avg_abs_sensitivity

    sensitivity_data = avg_abs_sensitivity if abs_sensitivity else avg_sensitivity
    channel = 'tuvw'.index(feature)
    axis = ['width', 'depth', 'height'].index(dim)
    
    visualization.show_latent_patterns(sensitivity_data, abs_sensitivity, num_patterns, channel, 
                                       slice, axis, cols=cols, contour=contour)

interactive(children=(Dropdown(description='trained_model', options=('D4cnn/hp_search/002_lr1e-4',), value='D4…